# FateCompass - basics using differentiation potential 

Here you will be guided through the basics of how to implement the FateCompass pipeline. 

First of all, FateCompass has two main inputs:

1. pre-preprocessed anndata object with annotated cell types and  
a. RNA velocities – in case you have a coherent profile, or  
b. **prior knowledge on initial and final fates**  
2. binding sites matrix. We provide pre-computed binding sites matrices for human and mouse using computationally predicted binding sites from the [SwissRegulon](https://swissregulon.unibas.ch/sr/) database.

This example is applied to *in vitro* pancreatic $\beta$ cell differentiation. **In this dataset, the RNA velocity profile was pointing towards the wrong direction; but we have prior knowledge about the initial (NKX6-1 progenitors) and final fates (SC-$\alpha$, SC-$\beta$, and SC-EC -cells)**. See [here](https://doi.org/10.1038/s41586-019-1168-5) for more details.  

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import scanpy.external as sce
import scvelo as scv
import cellrank as cr
import matplotlib.pyplot as plt

In [2]:
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2

%matplotlib inline

### Load data

1. Annotated AnnData object  

**FateCompass** expects annotations of at least the celltypes (clustering). For downstream pre-processing we advise to use the guidelines of [scanpy](https://scanpy.readthedocs.io/en/stable/api.html#).

In [3]:
adata = scv.read('../../Data/scrnaseq/pancreas/veres2019_stage5_timecourse.h5ad', cache=True)
adata

AnnData object with n_obs × n_vars = 25299 × 16129
    obs: 'CellBatch', 'CellDay', 'CellFlask', 'DetailedLabels', 'Labels', '_TrainFilter', '_Valid', 'barcode_sequence', 'n_genes', 'n_counts'
    var: '_Valid', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Labels_colors', 'hvg', 'neighbors', 'nodes_fatecompass', 'pca', 'umap'
    obsm: 'TSNE', 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    layers: 'Ms', 'Mu', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

2. Binding site matrix    

This consist of a table where the columns are the TFs or motifs and the rows are the genes. Hence, each entry $n_{ij}$ corresponds to the binding sites of the TF $j$ in the regulatory region of the gene $j$. It can be read as a pandas data frame where the index are the gene names and the columns the TF names.  

In [4]:
#filename = '../../Data/binding_sites/human/hg19_BSperFamily.csv'
filename = 'data/binding_sites/binding_sites_human.csv'
bs = pd.read_csv(filename, index_col=0, header=0,sep=';')
bs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9652 entries, FAM41C to RPS4Y2
Columns: 339 entries, AHR_ARNT2 to ZNF8
dtypes: float64(339)
memory usage: 25.0+ MB


### Data pre-processing for TF activity estimation

The scanpy functions normalize and filter the data as follows:  
- `sc.pp.normalize_total` normalize each cell such that they have the same total count.  
- `sc.pp.log1p` log normalize the gene expression.  
- `scv.pp.filter_genes_dispersion` filter to keep top n variable genes. We suggest using `n_top_genes=10000`.

In [ ]:
# SKIP this if your anndata object is already pre-processed!!!
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata) 
scv.pp.filter_genes_dispersion(adata, n_top_genes=10000)

### Nearest neighbor graph representing the phenotypic manifold

The **FateCompass** function `graph_fatecompass` builds a nearest neighbor graph in a reduced space.  
For more detailed information about the graph, follow the **advanced mode tutorial**.  

The inputs for the `graph_fatecompass` function are:  
1. **adata**: pre-proccessed and log normalized anndata object. 
2. **mode**: which drift will you use for the transition probabilities. In this notebook we used **'potential'**.
3. **basis**: dimensionality reduction method to compute the nearest neighbor graph.
4. **components**: number of components to compute the nearest neighbor graph. 
5. **n_neighbors**: number of nearest neighbors.  

The output is the neighbor graph ('distances_fatecompass' and 'indices_fatecompass') stored in the anndata object: `adata.obsm`.

In [ ]:
mode = 'potential'
basis = 'umap'
components = 10
n_neighbors = 50

In [ ]:
from fatecompass import graph_fatecompass
graph_fatecompass(adata, mode, basis, components, n_neighbors)

### Prior biological knowledge on progenitor cells and final fates 

The differentiation potential way for inferring dynamics relies on prior biological knowledge about the initial and final fates. This could be provided to the **FateCompass** function `diff_potential_driven_stochastic_simulations` in **three different flavors**; the flavor is specified to the function via the `mode` parameter. 

**1.** `cell_types:` in this mode, you will specify which cell type is the initial and which cell types are the final.  
a. `initial_fate = ['name_initial_cell_type']`  
b. `end_fates = ['name_final_cell_type_1',...]`  
**2.** `marker_genes:` in this mode, you will specify marker genes with a very clear distinctive expression profile in the initial and final fates.  
a. `initial_fate = ['marker_gene_initial_cell_type']`  
b. `end_fates = ['marker_gene_final_cell_type_1',...]`  
**3.** `prior_knowledge_indices:` in this mode, you will annotate the anndata object prior running the function with the indices of the cells that correspond to the initial and final fates.  
a. `adata.uns['nodes_fatecompass'][0]:` Index of the initial fate.  
b. `adata.uns['nodes_fatecompass'][1:]:` Indices of the final fates. 

In [ ]:
mode = 'prior_knowledge_indices'
initial_fate = []#'prog_nkx61'#'SOX9'
end_fates = []#['scbeta','ph','ec']#['NPTX2','GCG','ADRA2A']

### Cell-fate decision dynamics using differentiation potential as direction of differentiation

The **FateCompass** function `diff_potential_driven_stochastic_simulations` infers transition probabilities using a Markov process on a network, limiting the transitions to the observable states (cells). In this situation, the drift for the transitions is an energy gradient of the form:  

\begin{equation}
W_{i} = \sum_k \frac{Q_k}{(distance(x_i,x_k)+1)^{n}}  
\end{equation}  

where $W_i$ is the potential energy of a given state; $k$ are the indices of initial and final states; $Q_k$ are the attraction or repultion coefficients to end-states and initial-state, respectively; $distance(x_i,x_k)$ is the shortest path length between a state $i$ and a given initial or final state $k$. The values of $Q$ and $n$ are determined heuristically such that the potential energy gradient is high enough for the initial states to be phenotypically distinct from the final states. Then, using the transition probabilities, the **FateCompass** function `diff_potential_driven_stochastic_simulations` performs stochastic simulations using a Monte Carlo sampling algorithm.  

The inputs for the `rna_velocity_driven_stochastic_simulations` function are:  
1. **adata**: pre-proccessed and log normalized anndata object.  
2. **cell_types_key**: key of the observations clustering to consider.  
3. **mode**: way for providing prior biological knowledge.  
4. **initial_fate**: information about the initial fate, it'll depend on the selected `mode`. It can be `['name_initial_cell_type']`, `['marker_gene_initial_cell_type']`, or `[]`.  
5. **end_fates**: information about the final fate, it'll depend on the selected `mode`. It can be `['name_final_cell_type_1',...]`, `['marker_gene_final_cell_type_1',...]`, or `[]`.  
6. **numiter**: number of iterations for the Monte Carlo sampling algortihm. Default: 2000.
7. **numsimcells**: number of trajectories to simulate. Default: 1000.  

The output are the differentiation trajectories ('states' and 'num_trajectories') stored in the anndata object: `adata.uns`, and the differentiation potential also stored in the anndata object: `adata.obs['potential']`.  

In [ ]:
from fatecompass import diff_potential_driven_stochastic_simulations
diff_potential_driven_stochastic_simulations(adata, cell_types_key='Labels', mode='prior_knowledge_indices', 
                                             initial_fate=[], end_fates=[], numiter=2000, numsimcells=1000)
#~ 3min

The number of trajectories ending in the given fates can be checked in: `adata.uns['num_trajectories']`

In [ ]:
adata.uns['num_trajectories']

The differentiation potential can be visualized using the basic plotting functions of scanpy. 

In [ ]:
sc.pl.umap(adata, color='potential', frameon=False,cmap='RdBu_r')

The stochastic trajectories can be visualized using the **FateCompass** function `plot_sim_cell`. This will provide the trajectory of a given simulated cell. 

In [ ]:
mycell = adata.uns['states'][:,115]

In [ ]:
from fatecompass import plot_sim_cell
plot_sim_cell(adata, 'umap',mycell, color = 'Labels')

### TF activity estimation using a linear model of gene regulation

The FateCompass function `tf_activities` fits TF activities from the gene expression data using the following linear model of gene regulation:  

\begin{equation}
E_{gc} = \sum_f(N_{gf} * A_{fc}) + noise     
\end{equation}

where $E_{gc}$ is the gene expression matrix, $N_{gf}$ is the binding sites matrix –each entry represents the probability of binding of the TF $f$ in the promoter region of the gene $g$–, and $A_{fc}$ is the TF activity matrix. To account for the noise, the function implements a data diffusion regularization, where the main assumption is that the activity of similar cells should be similar. Then, the learned activities are diffused in the nearest neighbor graph. The diffusion time is fitted using a cross-validation scheme. Finally, the distribution of the TF activities is learned using a bootstrapping strategy.  
<br>

The inputs for the `tf_activities` function are: 
1. **adata**: pre-proccessed and log normalized anndata object
2. **bs**: binding sites matrix. Data frame with rows equal genes and columns equal TFs. 
3. **tolerance**: minimum difference between the test and training datasets used in the cross-validation scheme. Default 1e-2.  

The output are the TF activies and their distribution ('tf_activities' and 'tf_activities_distribution') stored in the anndata object: `adata.uns`  


In [ ]:
from fatecompass import tf_activities #13:07
tf_activities(adata, bs, tolerance = 1.5e-3)#~2h 

The TF activities can be visualized using the basic plotting functions of scanpy. For doing that, add to `adata.obs` the value of the TF activities you wish to visualize.  

To check the complete list of TF family names run: `list(bs.columns)`

In [ ]:
adata.obs['nkx6-1_act'] = adata.uns['tf_activities'].loc[:,'NOTO_VSX2_DLX2_DLX6_NKX6-1']
adata.obs['isl1_act'] = adata.uns['tf_activities'].loc[:,'ISL1']

In [ ]:
sc.pl.umap(adata, color=['nkx6-1_act','isl1_act'], frameon=False, cmap='RdBu_r')

The activity of **all** the TFs can be visualized using `clustermap` from the *seaborn* library as follows:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=adata.uns['tf_activities']
df['Labels']=adata.obs['Labels'].values

clusters = df['Labels']
keys = adata.obs['Labels'].cat.categories
values = adata.uns['Labels_colors']

lut1 = {keys[i]: values[i] for i in range(len(keys))}
col_colors1 = clusters.map(lut1)

df = df.drop(['Labels'], axis=1)

In [ ]:
sns.set(font_scale=1.0)
g=sns.clustermap(df.T,col_cluster=True,row_cluster=True, col_colors=col_colors1,
                 cmap='RdBu_r',z_score=0,yticklabels=True,figsize=(20, 30))
ax = g.ax_heatmap
g.cax.set_visible(False)
g.ax_cbar.set_visible(True,)
ax.set_xticks([])
ax.set_xticklabels([])
plt.show()

### Average gene expression and activity profiles over stochastic trajectories

The **FateCompass** function `avg_profiles_over_trajectories` uses the stochactic tracjectories previously generated (stored in `adata.uns['states']`) to approximate the expected value of the mean and the standard error of the mean (sem).  

The inputs for the `avg_profiles_over_trajectories` are:  
1. **adata**: pre-proccessed and log normalized anndata object.  
2. **cell_types_key**: key of the observations clustering to consider.  

The outputs are the **mean** and **sem** of the gene expression ('mean_E', and 'sem_E') and activity ('mean_A', and 'sem_S') profiles stored in the anndata object: `adata.uns`.

In [ ]:
from fatecompass import avg_profiles_over_trajectories

avg_profiles_over_trajectories(adata, cell_types_key='Labels')

In [ ]:
from fatecompass import plot_trajectory
plot_trajectory(adata, mode ='mRNA' ,variable=['NKX6-1'], cell_types_key='Labels', trajectory=['scbeta','ec','ph'])

In [ ]:
plot_trajectory(adata, mode ='activity' ,variable=['NOTO_VSX2_DLX2_DLX6_NKX6-1'], cell_types_key='Labels', trajectory=['scbeta','ec','ph'])

In [ ]:
plot_trajectory(adata, mode ='activity' ,variable=['ISL1'], cell_types_key='Labels', trajectory=['scbeta','ec','ph'])

### Differential TF activity analysis  

The **FateCompass** function `differential_tf_activity` computes three criteria to guide the identification of lineage-specific regulators. The criteria are:  

1. **z-score**: it can be interpreted as the number of standard deviations that the activity of a TF is away from its average of zero corrected by the precision of the estimation (a.k.a. error bar).  
2. **Variability over time**: it can be interpreted ad the rate of change of the TF activity over simulated time. FateCompass gets a proxy of this by computing the standard deviation of the TF activity over the stochastic trajectories.  
3. **Dynamic correlation**: this is the cross-correlation between the TF activity and the average gene expression profile of the TF transcript. 

The inputs for `differential_tf_activity` are:  
1. **adata**: pre-proccessed and log normalized anndata object.  
2. **cell_types_key**: key of the observations clustering to consider.  

The outputs are the 'z_score', 'std_tf_time', 'cross_corr' and 'time_lags' stored in the anndata object: `adata.uns`.

In [ ]:
from fatecompass import differential_tf_activity
differential_tf_activity(adata,'Labels')

To identify lineage-specific regulators we suggest to explore the **density distribution of the some of criteria described above**; this exploration will guide the setting of thresholds for filtering. Below we provide examples of how to do the exploration.  

The **FateCompass** function `ksdensity_fatecompass` allows to explore two criteria: z-score and variability over time:  

1. The **z-score** is computed TF-wise, by plotting the density distribution we can visualize what is the most common value and the range. We suggest to set a threshold for the z-score higher than or equal to the most common value.  
2. The **variability over time** is computed TF-wise for **every differentiation trajectory**. Similarly to the z-score, we can visualize the density distribution for the different trajectories. We suggest to set a threshold for the variability over time higher than or equal to the mean of the most common values for the different trajectories.  

The inputs for `ksdensity_fatecompass` are:  
1. **adata**: pre-proccessed and log normalized anndata object. 
2. **criteria**: criteria to plot `['z_score','std_tf_time']`.  
3. **cell_types_key**: key of the observations clustering to consider. 
4. **trajectory**: for which trajectories you wish to plot the density distribution. It must be a key from cell_types_key.  

The output is a plot with the density distribution and of the selected criteria, i.e., `['z_score','std_tf_time']`, and the variability over time for the selected trajectories. Also, a red vertical line indicating the suggested threshold for each criterion. 

In [ ]:
from fatecompass import ksdensity_fatecompass
ksdensity_fatecompass(adata, criterion=['std_tf_time','z_score'], cell_types_key='Labels', 
                      trajectory=['scbeta','ec','ph'])

To perform the **filtering and get the results in a data frame** the **FateCompass** function `get_df_differential_tf_activity` formats the results in a table.  

The inputs for the `get_df_differential_tf_activity` function are:  
1. **adata**: pre-proccessed and log normalized anndata object. 
2. **fates**: differentiation trajectories of interest. These keys must belong to the key of the observations clustering. 
3. **thresholds**: thresholds to perform the filtering and fate assignment in a dictionary format. Below there is an example of how to create the dictionary.  

The output is a pandas data frame that can be exported, where the indices are the motif families, and the columns are:  
- TFs: each TF beloging to the respective motif family.  
- std_tf_time_*trajectory*: variability over time of the respective TF in the indicatated differentiation trajectory. This column will be present for each *trajectory*.  
- max_cross_corr_*trajectory*: value of maximum cross correlation between TF activity and expression of the TF mRNA. This column will be present for each *trajectory*.  
- time_max_cross_corr_*trajectory*: time lag at wich the maximum cross correlation occurs. This column will be present for each *trajectory*.  
- z_score: value of z-score. 
- FateCompass_prediction: Name of the differentiation trajectory for which FateCompass pipeline predicts a role based.  

In [ ]:
thresholds = {'variability': 0.002, 'z_score': 1.5, 'correlation': 0.5}
thresholds

In [ ]:
from fatecompass import get_df_differential_tf_activity
df = get_df_differential_tf_activity(adata,fates=['scbeta','ec','ph'], 
                                     thresholds=thresholds)
df

In [ ]:
df.to_csv('output/human.txt', header=True, index=True, sep='\t')

In [ ]:
x = adata.uns['time_lags']['scbeta']['NOTO_VSX2_DLX2_DLX6_NKX6-1']['NKX6-1']
y = adata.uns['cross_corr']['scbeta']['NOTO_VSX2_DLX2_DLX6_NKX6-1']['NKX6-1']
plt.plot(x,y)